<a href="https://colab.research.google.com/github/dhan-t/CCMACLRL_EXAM_COM222-ML-/blob/main/Midterms_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [29]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv(test_url)
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'
sf=pd.read_csv(sample_submission_url)

In [30]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [32]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [33]:
df.dropna(inplace=True)

In [34]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [35]:
df.describe()

,id,model_year,milage,price
count,162610.000000,162610.000000,162610.000000,1.626100e+05
mean,94197.039137,2015.158822,71105.487086,4.081699e+04
std,54467.770815,5.699531,50157.874111,7.653857e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47026.250000,2012.000000,29000.000000,1.570000e+04
50%,94139.000000,2016.000000,64300.000000,2.850000e+04
75%,141421.750000,2019.000000,101000.000000,4.699900e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [36]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
125846,125846,Cadillac,CTS Luxury,2008,96600,Gasoline,270.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,5-Speed A/T,Silver,–,None reported,Yes,11500
14451,14451,Land,Rover Range Rover Sport Supercharged Dynamic,2018,57000,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Black,None reported,Yes,45000
78469,78469,Aston,Martin DBS Superleggera,2019,22770,Gasoline,5.2L V12 48V GDI DOHC Twin Turbo,8-Speed Automatic,Black,Black,None reported,Yes,72567
155104,155104,Ford,F-150 XL,2017,14700,E85 Flex Fuel,395.0HP 5.0L 8 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Silver,Black,At least 1 accident or damage reported,Yes,29000
74545,74545,BMW,X1 sDrive28i,2022,12800,Gasoline,228.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Blue,None reported,Yes,35000
55523,55523,Toyota,Highlander Limited Platinum,2016,135140,Gasoline,270.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes,24000
11804,11804,Lexus,RX 450h Base,2021,30450,Hybrid,308.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,CVT Transmission,Silver,Black,None reported,Yes,49995
115883,115883,GMC,Acadia SLE,2019,58500,Gasoline,310.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,9-Speed A/T,Gray,Black,None reported,Yes,35000
98058,98058,Jaguar,XK8 Base,2000,40000,Gasoline,290.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,5-Speed A/T,Green,Beige,None reported,Yes,30000
156921,156921,Mercedes-Benz,E-Class E500,2008,74000,Gasoline,302.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Black,None reported,Yes,19500


In [37]:
from datetime import datetime

current_year = datetime.now().year
df['car_age'] = current_year - df['model_year'].astype(str).str.split('-', expand=True).astype(int)

In [38]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car_age
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,17
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,22
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,22
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,7
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,3


In [39]:
def extract_luxury(brand):
    luxury_yes = [
    'Lexus',
    'Audi',
    'BMW',
    'Mercedes-Benz',
    'Jaguar',
    'Porsche',
    'Rolls-Royce',
    'Lamborghini',
    'Maserati',
    'Ferrari',
    'McLaren',
    'Bentley',
    'Tesla',
    'Cadillac',
    'McLaren',
    'Volvo',
    'Land Rover',
    'Genesis',

]
    Yes = "Yes"
    No = "No"
    for is_luxury in luxury_yes:
        if is_luxury.lower() in brand.lower():
            return Yes
    return No

# Apply the function to create a new column
df['is_luxury'] = df['brand'].apply(extract_luxury)

In [40]:
df['transmission'] = df['transmission'].replace({
    'A/T': 'A/T',
    'Transmission w/Dual Shift Mode': 'DCT',
    '7-Speed A/T': 'A/T',
    '8-Speed A/T': 'A/T',
    '10-Speed Automatic': 'A/T',
    '6-Speed A/T': 'A/T',
    '10-Speed A/T': 'A/T',
    '9-Speed A/T': 'A/T',
    '9-Speed Automatic': 'A/T',
    '5-Speed A/T': 'A/T',
    '8-Speed Automatic': 'A/T',
    'CVT Transmission': 'CVT',
    '5-Speed M/T': 'M/T',
    'M/T': 'M/T',
    '6-Speed M/T': 'M/T',
    '6-Speed Automatic': 'A/T',
    '4-Speed Automatic': 'A/T',
    '7-Speed M/T': 'M/T',
    'Automatic': 'A/T',
    '4-Speed A/T': 'A/T',
    '2-Speed A/T': 'A/T',
    '6-Speed Manual': 'M/T',
    'Transmission Overdrive Switch': 'A/T',
    '8-Speed Automatic with Auto-Shift': 'A/T',
    '7-Speed Automatic with Auto-Shift': 'A/T',
    'Automatic CVT': 'CVT',
    '1-Speed A/T': 'A/T',
    '6-Speed Electronically Controlled Automatic with O': 'A/T',
    'F': None,
    '8-Speed Manual': 'M/T',
    '6-Speed Automatic with Auto-Shift': 'A/T',
    '7-Speed Automatic': 'A/T',
    '7-Speed Manual': 'M/T',
    '7-Speed': None,
    'Single-Speed Fixed Gear': None,
    '8-SPEED AT': 'A/T',
    '10-Speed Automatic with Overdrive': 'A/T',
    '2-Speed Automatic': 'A/T',
    'CVT-F': 'CVT',
    '5-Speed Automatic': 'A/T',
    '–': None,
    '7-Speed DCT Automatic': 'DCT',
    'Manual': 'M/T',
    '2': None,
    '1-Speed Automatic': 'A/T',
    '6-Speed': None,
    'SCHEDULED FOR OR IN PRODUCTION': None,
    'Variable': None,
    '9-Speed Automatic with Auto-Shift': 'A/T',
    '6 Speed Mt': 'M/T',
    '6 Speed At/Mt': 'DCT',
    '8-SPEED A/T': 'A/T'
})

In [41]:
df['accident'] = df['accident'].replace({
    'None reported': 'No',
    'At least 1 accident or damage reported': 'Yes',
})

In [42]:
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,car_age,is_luxury
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,No,Yes,4200,17,No
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,Yes,Yes,4999,22,No
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,No,Yes,13900,22,No
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,DCT,Black,Black,No,Yes,45000,7,Yes
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Beige,No,Yes,97500,3,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,188527,Chevrolet,Camaro Z28,1999,110000,Gasoline,310.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,White,Gray,No,Yes,14500,25,No
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,DCT,White,Beige,No,Yes,27500,7,Yes
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,Yes,Yes,30000,6,Yes
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,A/T,White,Black,No,Yes,86900,3,Yes


In [43]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,92
ext_col,0
int_col,0


In [44]:
label_encoder = LabelEncoder()
df['brand'] = label_encoder.fit_transform(df['brand'])

In [45]:
drop_col=['id','ext_col','int_col', 'fuel_type', 'engine', 'model', 'clean_title']
df.drop(drop_col,inplace=True,axis=1)
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns]

,transmission,accident,is_luxury
0,A/T,No,No
1,A/T,Yes,No
2,A/T,No,No
3,DCT,No,Yes
4,A/T,No,Yes
...,...,...,...
188527,A/T,No,No
188528,DCT,No,Yes
188529,A/T,Yes,Yes
188530,A/T,No,Yes


In [46]:
for column in object_columns:
    unique_values = df[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'transmission':
['A/T' 'DCT' 'CVT' 'M/T' None]

Unique values in 'accident':
['No' 'Yes']

Unique values in 'is_luxury':
['No' 'Yes']



In [47]:
df = pd.get_dummies(df, columns=object_columns)

In [48]:
df

,brand,model_year,milage,price,car_age,transmission_A/T,transmission_CVT,transmission_DCT,transmission_M/T,accident_No,accident_Yes,is_luxury_No,is_luxury_Yes
0,31,2007,213000,4200,17,True,False,False,False,True,False,True,False
1,28,2002,143250,4999,22,True,False,False,False,False,True,True,False
2,9,2002,136731,13900,22,True,False,False,False,True,False,True,False
3,16,2017,19500,45000,7,False,False,True,False,True,False,False,True
4,36,2021,7388,97500,3,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,9,1999,110000,14500,25,True,False,False,False,True,False,True,False
188528,8,2017,49000,27500,7,False,False,True,False,True,False,False,True
188529,36,2018,28600,30000,6,True,False,False,False,False,True,False,True
188530,36,2021,13650,86900,3,True,False,False,False,True,False,False,True


In [49]:
from sklearn.preprocessing import MinMaxScaler
non_boolean_numerical_features = ["model_year",'milage','price','car_age']
scaler = MinMaxScaler()
df[non_boolean_numerical_features] = scaler.fit_transform(df[non_boolean_numerical_features])

In [50]:
df

,brand,model_year,milage,price,car_age,transmission_A/T,transmission_CVT,transmission_DCT,transmission_M/T,accident_No,accident_Yes,is_luxury_No,is_luxury_Yes
0,31,0.66,0.525809,0.000745,0.34,True,False,False,False,True,False,True,False
1,28,0.56,0.353544,0.001016,0.44,True,False,False,False,False,True,True,False
2,9,0.56,0.337444,0.004031,0.44,True,False,False,False,True,False,True,False
3,16,0.86,0.047913,0.014566,0.14,False,False,True,False,True,False,False,True
4,36,0.94,0.018000,0.032350,0.06,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,9,0.50,0.271425,0.004234,0.50,True,False,False,False,True,False,True,False
188528,8,0.86,0.120771,0.008638,0.14,False,False,True,False,True,False,False,True
188529,36,0.88,0.070388,0.009485,0.12,True,False,False,False,False,True,False,True
188530,36,0.94,0.033465,0.028759,0.06,True,False,False,False,True,False,False,True


In [51]:
df

,brand,model_year,milage,price,car_age,transmission_A/T,transmission_CVT,transmission_DCT,transmission_M/T,accident_No,accident_Yes,is_luxury_No,is_luxury_Yes
0,31,0.66,0.525809,0.000745,0.34,True,False,False,False,True,False,True,False
1,28,0.56,0.353544,0.001016,0.44,True,False,False,False,False,True,True,False
2,9,0.56,0.337444,0.004031,0.44,True,False,False,False,True,False,True,False
3,16,0.86,0.047913,0.014566,0.14,False,False,True,False,True,False,False,True
4,36,0.94,0.018000,0.032350,0.06,True,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188527,9,0.50,0.271425,0.004234,0.50,True,False,False,False,True,False,True,False
188528,8,0.86,0.120771,0.008638,0.14,False,False,True,False,True,False,False,True
188529,36,0.88,0.070388,0.009485,0.12,True,False,False,False,False,True,False,True
188530,36,0.94,0.033465,0.028759,0.06,True,False,False,False,True,False,False,True


In [52]:
# x = df.drop(['brand_Acura', 'brand_Alfa',        'brand_Aston', 'brand_Audi', 'brand_BMW', 'brand_Bentley',        'brand_Bugatti', 'brand_Buick', 'brand_Cadillac', 'brand_Chevrolet',        'brand_Chrysler', 'brand_Dodge', 'brand_FIAT', 'brand_Ferrari',        'brand_Ford', 'brand_GMC', 'brand_Genesis', 'brand_Honda',        'brand_Hummer', 'brand_Hyundai', 'brand_INFINITI', 'brand_Jaguar',        'brand_Jeep', 'brand_Karma', 'brand_Kia', 'brand_Lamborghini',        'brand_Land', 'brand_Lexus', 'brand_Lincoln', 'brand_Lotus',        'brand_Lucid', 'brand_MINI', 'brand_Maserati', 'brand_Maybach',        'brand_Mazda', 'brand_McLaren', 'brand_Mercedes-Benz', 'brand_Mercury',        'brand_Mitsubishi', 'brand_Nissan', 'brand_Plymouth', 'brand_Polestar',        'brand_Pontiac', 'brand_Porsche', 'brand_RAM', 'brand_Rivian',        'brand_Rolls-Royce', 'brand_Saab', 'brand_Saturn', 'brand_Scion',        'brand_Subaru', 'brand_Suzuki', 'brand_Tesla', 'brand_Toyota', 'brand_Volkswagen', 'brand_Volvo', 'brand_smart'],axis=1)
# x

In [53]:
x = df.drop(['price'],axis=1)
y = df['price']

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [55]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [56]:
y_pred_train = linear_reg.predict(X_train)
y_pred_test = linear_reg.predict(X_test)

In [57]:
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Training MSE: {train_mse}")
print(f"Testing MSE: {test_mse}")
print(f"Training R2 Score: {train_r2}")
print(f"Testing R2 Score: {test_r2}")

Training MSE: 0.0005852110441183176
Testing MSE: 0.0007251596504868553
Training R2 Score: 0.09055022740662444
Testing R2 Score: 0.07863572228512661


In [58]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

0.026928788507596387

In [59]:
dt['car_age'] = current_year - dt['model_year'].astype(str).str.split('-', expand=True).astype(int)
dt['transmission_A/T'] = df['transmission_A/T'].astype(int)
dt['transmission_CVT'] = df['transmission_CVT'].astype(int)
dt['transmission_DCT'] = df['transmission_DCT'].astype(int)
dt['transmission_M/T'] = df['transmission_M/T'].astype(int)
dt['accident_No'] = df['accident_No'].astype(int)
dt['accident_Yes'] = df['accident_Yes'].astype(int)
dt['is_luxury_No'] = df['is_luxury_No'].astype(int)
dt['is_luxury_Yes'] = df['is_luxury_Yes'].astype(int)


dt.drop(columns=['accident','clean_title','engine', 'ext_col', 'fuel_type', 'id', 'int_col', 'model', 'transmission'], axis=1, inplace=True)

In [60]:
label_encoder = LabelEncoder()
dt['brand'] = label_encoder.fit_transform(dt['brand'])

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   brand             162610 non-null  int64  
 1   model_year        162610 non-null  float64
 2   milage            162610 non-null  float64
 3   price             162610 non-null  float64
 4   car_age           162610 non-null  float64
 5   transmission_A/T  162610 non-null  bool   
 6   transmission_CVT  162610 non-null  bool   
 7   transmission_DCT  162610 non-null  bool   
 8   transmission_M/T  162610 non-null  bool   
 9   accident_No       162610 non-null  bool   
 10  accident_Yes      162610 non-null  bool   
 11  is_luxury_No      162610 non-null  bool   
 12  is_luxury_Yes     162610 non-null  bool   
dtypes: bool(8), float64(4), int64(1)
memory usage: 12.7 MB


In [62]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   brand             125690 non-null  int64  
 1   model_year        125690 non-null  int64  
 2   milage            125690 non-null  int64  
 3   car_age           125690 non-null  int64  
 4   transmission_A/T  108458 non-null  float64
 5   transmission_CVT  108458 non-null  float64
 6   transmission_DCT  108458 non-null  float64
 7   transmission_M/T  108458 non-null  float64
 8   accident_No       108458 non-null  float64
 9   accident_Yes      108458 non-null  float64
 10  is_luxury_No      108458 non-null  float64
 11  is_luxury_Yes     108458 non-null  float64
dtypes: float64(8), int64(4)
memory usage: 11.5 MB


In [63]:
dt.fillna(0, inplace=True)

In [64]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'class': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
